
# API Practice - Paired Lab

## Introduction

For some further practice with APIs (and with SQL!), we're going to work on a single big lab to apply what we've learned in Module 1!

![](https://media.tenor.com/images/faa7904870d4661b3f077f1c49fbbb46/tenor.gif)

## The Goal

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this [kaggle page](https://www.kaggle.com/laudanum/footballdelphi). Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team and calculate some summary statistics. 

Unlike previous labs, this lab is more open-ended. At minimum, you'll need to:

* Query the SQL database
* Calculate summary statistics
* Get the weather data from the DarkSky API

Upon completion of this lab, you should be able to see/access the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* The team's win percentage on days where it was raining (in Berlin) during games in the 2011 season. 

![](https://media.giphy.com/media/4TkcwHdT1LSLw2rrEN/giphy.gif)

## The Data

You'll find a database containing information about soccer teams and the matches they've played in the file `database.sqlite`. 

### Getting the Weather Data

You'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the [DarkSky API](https://darksky.net/dev) to get the historical weather data for that day. 

Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely German, so go ahead and just use the weather in Berlin, Germany as a proxy for this information. If it was raining in Berlin on the day the game was played, count that as rain game-- **_you do not need to try and figure out the actual weather at each game's location, because we don't have that information!_**

#### NOTE: The DarkSky API is limited to 1000 free API calls a day, so be sure to test your model on very small samples. Otherwise, you'll hit the rate limit!

### Some Advice

This is a paired afternoon lab, meant to take just one afternoon. You may run out of time, and that's okay! But do make sure to comment your code *while you're writing it* in case you don't have time today and want to go back later.

ALSO: It's okay to use external tools to make your life easier -- that's what they're for! Using tools like the [DB Browser for SQLite](https://sqlitebrowser.org/) to visualize and interact with your SQL database, or [Postman](https://www.getpostman.com/) to test your API query, is one aspect of working smarter (not harder)! 

![](https://media2.giphy.com/media/11F0d3IVhQbreE/giphy.gif)


In [1]:
import sqlite3
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
matches_df = pd.DataFrame(cur.execute('''SELECT * FROM Matches''').fetchall())
matches_df.columns = [x[0] for x in cur.description]
matches_df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A


In [4]:
matches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24625 entries, 0 to 24624
Data columns (total 9 columns):
Match_ID    24625 non-null int64
Div         24625 non-null object
Season      24625 non-null int64
Date        24625 non-null object
HomeTeam    24625 non-null object
AwayTeam    24625 non-null object
FTHG        24625 non-null int64
FTAG        24625 non-null int64
FTR         24625 non-null object
dtypes: int64(4), object(5)
memory usage: 1.7+ MB


In [5]:
matches_df_2011 = matches_df[matches_df['Season'] == 2011]
matches_df_2011.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
1091,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1092,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
1093,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
1094,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
1095,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [6]:
home_matches_df_2011 = matches_df_2011.groupby(['HomeTeam','FTR'])['Match_ID'].count()

In [7]:
home_matches_df_2011.head()

HomeTeam  FTR
Aachen    A      7
          D      6
          H      4
Arsenal   A      3
          D      4
Name: Match_ID, dtype: int64

In [8]:
home_matches_df_2011 = home_matches_df_2011.drop(labels = ['A','D'], level = 1)

In [9]:
home_matches_df_2011 = home_matches_df_2011.reset_index(level = 1)

In [10]:
home_matches_df_2011.head()

,FTR,Match_ID
HomeTeam,,
Aachen,H,4
Arsenal,H,12
Aston Villa,H,4
Augsburg,H,6
Bayern Munich,H,14


In [11]:
away_matches_df_2011 = matches_df_2011.groupby(['AwayTeam','FTR'])['Match_ID'].count()

In [12]:
away_matches_df_2011.head()

AwayTeam  FTR
Aachen    A      2
          D      7
          H      8
Arsenal   A      9
          D      3
Name: Match_ID, dtype: int64

In [13]:
away_matches_df_2011 = away_matches_df_2011.drop(labels = ['H','D'], level = 1)

In [14]:
away_matches_df_2011 = away_matches_df_2011.reset_index(level = 1)

In [15]:
away_matches_df_2011

,FTR,Match_ID
AwayTeam,,
Aachen,A,2
Arsenal,A,9
Aston Villa,A,3
Augsburg,A,2
Bayern Munich,A,9
Blackburn,A,2
Bochum,A,3
Bolton,A,6
Braunschweig,A,4


In [16]:
total_wins_df = pd.merge(home_matches_df_2011, away_matches_df_2011, left_on = home_matches_df_2011.index, right_on = away_matches_df_2011.index)

In [17]:
total_wins_df['Wins'] = total_wins_df['Match_ID_x'] + total_wins_df['Match_ID_y']

In [18]:
total_wins_df.head()

,key_0,FTR_x,Match_ID_x,FTR_y,Match_ID_y,Wins
0,Aachen,H,4,A,2,6
1,Arsenal,H,12,A,9,21
2,Aston Villa,H,4,A,3,7
3,Augsburg,H,6,A,2,8
4,Bayern Munich,H,14,A,9,23


In [19]:
total_wins_df.drop(labels = [ 'FTR_x', 'FTR_y', 'Match_ID_x', 'Match_ID_y'], axis = 'columns', inplace = True)

In [20]:
total_wins_df.rename(columns = {'key_0':'Team'}, inplace = True)

In [21]:
total_wins_df.head(10)

,Team,Wins
0,Aachen,6
1,Arsenal,21
2,Aston Villa,7
3,Augsburg,8
4,Bayern Munich,23
5,Blackburn,8
6,Bochum,10
7,Bolton,10
8,Braunschweig,10
9,Chelsea,18


In [22]:
home_games_played_df_2011 = matches_df_2011.groupby('HomeTeam')['Match_ID'].count()

In [23]:
away_games_played_df_2011 = matches_df_2011.groupby('AwayTeam')['Match_ID'].count()

In [24]:
total_games_played_df = pd.merge(home_games_played_df_2011, away_games_played_df_2011, left_on = home_games_played_df_2011.index, right_on = away_games_played_df_2011.index)

In [25]:
total_games_played_df.head()

,key_0,Match_ID_x,Match_ID_y
0,Aachen,17,17
1,Arsenal,19,19
2,Aston Villa,19,19
3,Augsburg,17,17
4,Bayern Munich,17,17


In [26]:
total_games_played_df['Games Played'] = total_games_played_df['Match_ID_x'] + total_games_played_df['Match_ID_y']

In [27]:
total_games_played_df.drop(labels = ['Match_ID_x', 'Match_ID_y'], axis = 'columns', inplace = True)

In [28]:
total_games_played_df.rename(columns = {'key_0': 'Team'}, inplace = True)

In [29]:
home_goals_scored_df_2011 = matches_df_2011.groupby('HomeTeam')['FTHG'].sum()

In [30]:
home_goals_scored_df_2011.head()

HomeTeam
Aachen           15
Arsenal          39
Aston Villa      20
Augsburg         20
Bayern Munich    49
Name: FTHG, dtype: int64

In [31]:
away_goals_scored_df_2011 = matches_df_2011.groupby('AwayTeam')['FTAG'].sum()

In [32]:
away_goals_scored_df_2011.head()

AwayTeam
Aachen           15
Arsenal          35
Aston Villa      17
Augsburg         16
Bayern Munich    28
Name: FTAG, dtype: int64

In [33]:
total_goals_scored_df = pd.merge(home_goals_scored_df_2011, away_goals_scored_df_2011, left_on = home_goals_scored_df_2011.index, right_on = away_goals_scored_df_2011.index)

In [34]:
total_goals_scored_df.head()

,key_0,FTHG,FTAG
0,Aachen,15,15
1,Arsenal,39,35
2,Aston Villa,20,17
3,Augsburg,20,16
4,Bayern Munich,49,28


In [35]:
total_goals_scored_df['Goals Scored'] = total_goals_scored_df['FTHG'] + total_goals_scored_df['FTAG']

In [36]:
total_goals_scored_df.head()

,key_0,FTHG,FTAG,Goals Scored
0,Aachen,15,15,30
1,Arsenal,39,35,74
2,Aston Villa,20,17,37
3,Augsburg,20,16,36
4,Bayern Munich,49,28,77


In [37]:
total_goals_scored_df.drop(labels = ['FTHG', 'FTAG'], axis = 'columns', inplace = True)

In [38]:
total_goals_scored_df.rename(columns = {'key_0': 'Team'}, inplace = True)

In [39]:
total_goals_scored_df.head()

,Team,Goals Scored
0,Aachen,30
1,Arsenal,74
2,Aston Villa,37
3,Augsburg,36
4,Bayern Munich,77


In [40]:
total_games_played_df.head()

,Team,Games Played
0,Aachen,34
1,Arsenal,38
2,Aston Villa,38
3,Augsburg,34
4,Bayern Munich,34


In [41]:
total_wins_df.head()

,Team,Wins
0,Aachen,6
1,Arsenal,21
2,Aston Villa,7
3,Augsburg,8
4,Bayern Munich,23


In [42]:
combined_df = pd.merge(total_wins_df, total_games_played_df, left_on = total_wins_df.index, right_on = total_games_played_df.index)

In [43]:
combined_df.head()

,key_0,Team_x,Wins,Team_y,Games Played
0,0,Aachen,6,Aachen,34
1,1,Arsenal,21,Arsenal,38
2,2,Aston Villa,7,Aston Villa,38
3,3,Augsburg,8,Augsburg,34
4,4,Bayern Munich,23,Bayern Munich,34


In [44]:
combined_df.drop(labels = ['key_0', 'Team_y'], axis = 'columns', inplace = True)

In [45]:
combined_df = pd.merge(combined_df, total_goals_scored_df, left_on = combined_df.index, right_on = total_goals_scored_df.index)

In [46]:
combined_df.head()

,key_0,Team_x,Wins,Games Played,Team,Goals Scored
0,0,Aachen,6,34,Aachen,30
1,1,Arsenal,21,38,Arsenal,74
2,2,Aston Villa,7,38,Aston Villa,37
3,3,Augsburg,8,34,Augsburg,36
4,4,Bayern Munich,23,34,Bayern Munich,77


In [47]:
combined_df.drop(labels = ['key_0', 'Team'], axis = 'columns', inplace = True)

In [48]:
combined_df.rename(columns = {'Team_x': 'Team'}, inplace = True)

In [49]:
combined_df.head()

,Team,Wins,Games Played,Goals Scored
0,Aachen,6,34,30
1,Arsenal,21,38,74
2,Aston Villa,7,38,37
3,Augsburg,8,34,36
4,Bayern Munich,23,34,77


## Playing with Darksky now

In [50]:
import requests

In [51]:
# Need to import json to do this
import json

# Define a function to open the json
def get_keys(path):
    with open(path) as f:
        return json.load(f)

# Using the function to open and load all keys in that file 
api_keys = get_keys("/Users/anupj/Documents/APIs/api.json")

# Setting the first (and only) value as a variable
ds_key_1 = list(api_keys.values())[0]

In [52]:
berlin_latitude = 52.5200
berlin_longitude = 13.4050

In [58]:
matches_df_2011.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
1091,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1092,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
1093,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
1094,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
1095,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [59]:
match_dates_series = pd.Series(matches_df_2011['Date'].unique())

In [60]:
match_dates_df = match_dates_series.to_frame()

In [61]:
match_dates_df.head()

,0
0,2012-03-31
1,2011-12-11
2,2011-08-13
3,2011-11-27
4,2012-02-18


In [62]:
len(match_dates_df)

165

In [63]:
match_dates_df.rename(columns = {0: 'Date'}, inplace = True)

In [82]:
test_df = match_dates_df.head(30)

In [83]:
test_df['Weather'] = "not filled"

C:\Users\anupj\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [84]:
for i in range(len(test_df)):
    api_call = requests.get('https://api.darksky.net/forecast/{}/{},{},{}T00:00:00?exclude=currently,hourly,flags'.format(ds_key_1, berlin_latitude, berlin_longitude,test_df['Date'][i])).json()['daily']['data'][0]
    if api_call['precipIntensity'] == 0:
        test_df['Weather'][i] = 'No Rain'
    else:
        test_df['Weather'][i] = 'Rain'

In [85]:
test_df.head(30)

,Date,Weather
0,2012-03-31,Rain
1,2011-12-11,No Rain
2,2011-08-13,No Rain
3,2011-11-27,No Rain
4,2012-02-18,No Rain
5,2012-01-20,Rain
6,2012-02-04,No Rain
7,2012-04-21,No Rain
8,2011-09-18,Rain
9,2011-10-23,Rain


In [86]:
match_dates_df['Weather'] = 'not filled'

In [91]:
match_dates_df.head()

,Date,Weather
0,2012-03-31,Rain
1,2011-12-11,No Rain
2,2011-08-13,No Rain
3,2011-11-27,No Rain
4,2012-02-18,No Rain


In [88]:
for i in range(len(match_dates_df)):
    api_call = requests.get('https://api.darksky.net/forecast/{}/{},{},{}T00:00:00?exclude=currently,hourly,flags'.format(ds_key_1, berlin_latitude, berlin_longitude,match_dates_df['Date'][i])).json()['daily']['data'][0]
    if api_call['precipIntensity'] == 0:
        match_dates_df['Weather'][i] = 'No Rain'
    else:
        match_dates_df['Weather'][i] = 'Rain'

In [90]:
match_dates_df.head()

,Date,Weather
0,2012-03-31,Rain
1,2011-12-11,No Rain
2,2011-08-13,No Rain
3,2011-11-27,No Rain
4,2012-02-18,No Rain


In [96]:
matches_with_weather_df_2011 = pd.merge(matches_df_2011, match_dates_df, on = 'Date')

In [97]:
matches_with_weather_df_2011

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Weather
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,Rain
1,1189,D1,2011,2012-03-31,Augsburg,FC Koln,2,1,H,Rain
2,1206,D1,2011,2012-03-31,Leverkusen,Freiburg,0,2,A,Rain
3,1215,D1,2011,2012-03-31,Kaiserslautern,Hamburg,0,1,A,Rain
4,1396,D2,2011,2012-03-31,Munich 1860,Hansa Rostock,0,1,A,Rain
...,...,...,...,...,...,...,...,...,...,...
987,44870,E0,2011,2012-05-13,Sunderland,Man United,0,1,A,No Rain
988,44871,E0,2011,2012-05-13,Swansea,Liverpool,1,0,H,No Rain
989,44872,E0,2011,2012-05-13,Tottenham,Fulham,2,0,H,No Rain
990,44873,E0,2011,2012-05-13,West Brom,Arsenal,2,3,A,No Rain


In [112]:
home_wins_rain_df = matches_with_weather_df_2011.groupby(['HomeTeam','Weather','FTR'])['Match_ID'].count()

In [115]:
home_wins_rain_df.head(10)

HomeTeam     Weather  FTR
Aachen       Rain     D      4
                      H      1
Arsenal      Rain     A      1
                      D      2
                      H      6
Aston Villa  Rain     A      6
                      D      2
                      H      1
Augsburg     Rain     A      2
                      D      1
Name: Match_ID, dtype: int64

In [114]:
home_wins_rain_df = home_wins_rain_df.drop(labels = ['No Rain'],level = 1)

In [117]:
home_wins_rain_df = home_wins_rain_df.drop(labels = ['A','D'],level = 2)

In [132]:
home_wins_rain_df.head()

,,,Match_ID
HomeTeam,Weather,FTR,
Aachen,Rain,H,1
Arsenal,Rain,H,6
Aston Villa,Rain,H,1
Augsburg,Rain,H,4
Bayern Munich,Rain,H,3


In [130]:
home_wins_rain_df = home_wins_rain_df.to_frame()

In [131]:
type(home_wins_rain_df)

pandas.core.frame.DataFrame

In [152]:
home_wins_rain_df.reset_index(inplace = True)

In [158]:
home_wins_rain_df.head()

,HomeTeam,Match_ID
0,Aachen,1
1,Arsenal,6
2,Aston Villa,1
3,Augsburg,4
4,Bayern Munich,3


In [155]:
type(home_wins_rain_df)

pandas.core.frame.DataFrame

In [157]:
home_wins_rain_df.drop(labels = ['Weather', 'FTR'], axis = 1, inplace = True)

In [166]:
away_wins_rain_df = matches_with_weather_df_2011.groupby(['AwayTeam','Weather','FTR'])['Match_ID'].count()

In [167]:
away_wins_rain_df.head()

AwayTeam  Weather  FTR
Aachen    No Rain  D      2
                   H      7
          Rain     A      2
                   D      5
                   H      1
Name: Match_ID, dtype: int64

In [168]:
away_wins_rain_df = away_wins_rain_df.drop(labels = ['No Rain'],level = 1)

In [169]:
away_wins_rain_df = away_wins_rain_df.drop(labels = ['H','D'],level = 2)

In [170]:
away_wins_rain_df = away_wins_rain_df.to_frame()

In [171]:
away_wins_rain_df.reset_index(inplace = True)

In [172]:
away_wins_rain_df.drop(labels = ['Weather', 'FTR'], axis = 1, inplace = True)

In [173]:
away_wins_rain_df.head()

,AwayTeam,Match_ID
0,Aachen,2
1,Arsenal,2
2,Aston Villa,1
3,Bayern Munich,5
4,Bochum,2


In [174]:
rainy_wins_df = pd.merge(home_wins_rain_df, away_wins_rain_df, left_on = home_wins_rain_df.index, right_on = away_wins_rain_df.index)

In [177]:
rainy_wins_df.head()

,key_0,HomeTeam,Match_ID_x,AwayTeam,Match_ID_y,Rainy Wins
0,0,Aachen,1,Aachen,2,3
1,1,Arsenal,6,Arsenal,2,8
2,2,Aston Villa,1,Aston Villa,1,2
3,3,Augsburg,4,Bayern Munich,5,9
4,4,Bayern Munich,3,Bochum,2,5


In [176]:
rainy_wins_df['Rainy Wins'] = rainy_wins_df['Match_ID_x'] + rainy_wins_df['Match_ID_y']

In [178]:
rainy_wins_df.columns

Index(['key_0', 'HomeTeam', 'Match_ID_x', 'AwayTeam', 'Match_ID_y',
       'Rainy Wins'],
      dtype='object')

In [180]:
rainy_wins_df.drop(labels = ['key_0', 'Match_ID_x', 'AwayTeam', 'Match_ID_y'], axis = 1, inplace = True)

In [181]:
combined_df.head()

,Team,Wins,Games Played,Goals Scored
0,Aachen,6,34,30
1,Arsenal,21,38,74
2,Aston Villa,7,38,37
3,Augsburg,8,34,36
4,Bayern Munich,23,34,77


In [182]:
final_df = rainy_wins_df = pd.merge(combined_df, rainy_wins_df, left_on = combined_df.index, right_on = rainy_wins_df.index)

In [183]:
final_df.head()

,key_0,Team,Wins,Games Played,Goals Scored,HomeTeam,Rainy Wins
0,0,Aachen,6,34,30,Aachen,3
1,1,Arsenal,21,38,74,Arsenal,8
2,2,Aston Villa,7,38,37,Aston Villa,2
3,3,Augsburg,8,34,36,Augsburg,9
4,4,Bayern Munich,23,34,77,Bayern Munich,5


In [185]:
final_df.drop(labels = ['key_0', 'HomeTeam'], axis = 1, inplace = True)

In [186]:
final_df.head()

,Team,Wins,Games Played,Goals Scored,Rainy Wins
0,Aachen,6,34,30,3
1,Arsenal,21,38,74,8
2,Aston Villa,7,38,37,2
3,Augsburg,8,34,36,9
4,Bayern Munich,23,34,77,5


In [191]:
rainy_home_games = matches_with_weather_df_2011.groupby(['HomeTeam', 'Weather'])['Match_ID'].count()

In [192]:
rainy_home_games.head()

HomeTeam     Weather
Aachen       No Rain    12
             Rain        5
Arsenal      No Rain    10
             Rain        9
Aston Villa  No Rain    10
Name: Match_ID, dtype: int64

In [197]:
rainy_home_games.drop(labels = ['No Rain'], level = 1, inplace = True)

In [203]:
rainy_home_games.reset_index()

,HomeTeam,Weather,Match_ID
0,Aachen,Rain,5
1,Arsenal,Rain,9
2,Aston Villa,Rain,9
3,Augsburg,Rain,7
4,Bayern Munich,Rain,4
5,Blackburn,Rain,7
6,Bochum,Rain,5
7,Bolton,Rain,6
8,Braunschweig,Rain,4
9,Chelsea,Rain,4


In [204]:
rainy_total_games = matches_with_weather_df_2011.groupby(['AwayTeam', 'Weather'])['Match_ID'].count()

In [205]:
rainy_total_games.drop(labels = ['No Rain'], level = 1, inplace = True)

In [210]:
rainy_home_games = rainy_home_games.reset_index()

In [211]:
rainy_home_games.head()

,HomeTeam,Weather,Match_ID
0,Aachen,Rain,5
1,Arsenal,Rain,9
2,Aston Villa,Rain,9
3,Augsburg,Rain,7
4,Bayern Munich,Rain,4


In [214]:
rainy_total_games = rainy_total_games.reset_index()

In [215]:
rainy_total_games_df = pd.merge(rainy_total_games, rainy_home_games, left_on = rainy_total_games.index, right_on = rainy_home_games.index)

In [216]:
rainy_total_games_df.head()

,key_0,AwayTeam,Weather_x,Match_ID_x,HomeTeam,Weather_y,Match_ID_y
0,0,Aachen,Rain,8,Aachen,Rain,5
1,1,Arsenal,Rain,5,Arsenal,Rain,9
2,2,Aston Villa,Rain,3,Aston Villa,Rain,9
3,3,Augsburg,Rain,4,Augsburg,Rain,7
4,4,Bayern Munich,Rain,7,Bayern Munich,Rain,4


In [218]:
rainy_total_games_df['Total Rainy Games'] = rainy_total_games_df['Match_ID_x'] + rainy_total_games_df['Match_ID_y']

In [219]:
rainy_total_games_df.columns

Index(['key_0', 'AwayTeam', 'Weather_x', 'Match_ID_x', 'HomeTeam', 'Weather_y',
       'Match_ID_y', 'Total Rainy Games'],
      dtype='object')

In [221]:
rainy_total_games_df.drop(labels = ['key_0', 'Weather_x', 'Match_ID_x', 'HomeTeam', 'Weather_y', 'Match_ID_y'], axis = 1, inplace = True)

In [222]:
final_final_df = pd.merge(final_df, rainy_total_games_df, left_on = final_df.index, right_on = rainy_total_games_df.index)

In [223]:
final_final_df.head()

,key_0,Team,Wins,Games Played,Goals Scored,Rainy Wins,AwayTeam,Total Rainy Games
0,0,Aachen,6,34,30,3,Aachen,13
1,1,Arsenal,21,38,74,8,Arsenal,14
2,2,Aston Villa,7,38,37,2,Aston Villa,12
3,3,Augsburg,8,34,36,9,Augsburg,11
4,4,Bayern Munich,23,34,77,5,Bayern Munich,11


In [224]:
final_final_df.columns

Index(['key_0', 'Team', 'Wins', 'Games Played', 'Goals Scored', 'Rainy Wins',
       'AwayTeam', 'Total Rainy Games'],
      dtype='object')

In [227]:
final_final_df.drop(labels = ['key_0', 'AwayTeam'], axis = 1, inplace = True)

In [228]:
final_final_df['Win %'] = (final_final_df['Wins']/final_final_df['Games Played']) * 100

In [229]:
final_final_df['Rainy Win %'] = (final_final_df['Rainy Wins']/final_final_df['Total Rainy Games']) * 100

In [230]:
final_final_df.head()

,Team,Wins,Games Played,Goals Scored,Rainy Wins,Total Rainy Games,Win %,Rainy Win %
0,Aachen,6,34,30,3,13,17.647059,23.076923
1,Arsenal,21,38,74,8,14,55.263158,57.142857
2,Aston Villa,7,38,37,2,12,18.421053,16.666667
3,Augsburg,8,34,36,9,11,23.529412,81.818182
4,Bayern Munich,23,34,77,5,11,67.647059,45.454545
